In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.sparse as sps
import sys

sys.path.append("../")

In [2]:
from contraction_utils import adaptive_contraction, shots_asarray

## Motivation
Precontraction is a great efficiency booster for covariance and spooktroscopy.
In principle there's no absolute upper bound to number of shots, because a last resort is looping through the shots to accumulate, but that's the least efficient way.
When it comes to memory, there are (at least) three possible limitations. From small to large these limitations are:

1. Contracting with `numpy.dot` or `@`
2. Instantiating `numpy.ndarray`s that includes all the shots
3. Instantiating a list or dictionary that includes all the shots

2 is a tighter limitation than 3 because `numpy.array` **allocates contiguous memory** to facilitate efficient operations, while the arrays in a list or a dict is not necessarily contiguous in memory.

Therefore we want this contraction function to:
1. Accept two tensors and contract over the 0th iterable "axis"
2. Accept tensors in three types: `numpy.ndarray`, `list` and `dict`
3. Accept optional slicing on each single-shot array.
4. In addition, for backward compatibility, accept `list` or `dict` of sparse matrices

In [3]:
# A Dummy dataset in three formats
np.random.seed(802) # This notebook is created on Aug02
Nshot = 5000

# np.array 
arrdum = {}
arrdum['A'] = np.random.randn(Nshot, 20)
arrdum['B'] = np.random.randn(Nshot, 100)
arrdum['C'] = np.random.randn(Nshot, 100,100)
arrdum['C'][arrdum['C']<0.8] = 0  # This sparsifies the (100,100) arrays to ~20% survival rate

# list
lstdum = {k: [ar for ar in arrdum[k]] for k in arrdum.keys()}
lstdum['C'] = [sps.coo_matrix(ar) for ar in lstdum['C']]

# dictionary
dctdum = {k: {i:ar for i,ar in enumerate(lstdum[k])} for k in lstdum.keys()}

In [4]:
slca = np.s_[10:14:2]
slcb = (30,50)
slcv = np.s_[:200, :200]
selecti = range(10,15)
selectk = list(range(10,15))

### Sanity checks on `adaptive_contraction`
The most common situation is: `A, B` are just two stacked-up matrices

In [11]:
res = adaptive_contraction(arrdum['A'], arrdum['B'])
check = arrdum['A'].T @ arrdum['B']
print(np.allclose(res, check), res.shape)

Chunk up by 2000 shots
True (20, 100)


The single shot spectra can be sliced with `a_slice, b_slice`. 
This may not seem helpful at small scale, but when it comes to memory limitations, cropping into the ROI **prior to** contraction matters, and that's what `adaptive_contraction` does.

In [10]:
res = adaptive_contraction(arrdum['A'], arrdum['B'], a_slice=slca)
check = arrdum['A'][:,slca].T @ arrdum['B']
print(np.allclose(res, check), res.shape)

Chunk up by 2000 shots
True (2, 100)


Slicing `numpy.ndarray` is really convenient, but what if one of the tensors is too big to be stacked up into an array? This is a common situation when dealing with 5e4-shot 500x500 VMI images. In that case, slicing with `a_slice, b_slice` becomes more useful. Plus, accepting list/dictionary of sparse matrices also becomes useful.

In [9]:
tmp = np.tensordot(arrdum['A'], arrdum['C'][:,:200,:200], axes=(0,0))
res = adaptive_contraction(dctdum['A'], dctdum['C'], b_slice=slcv, keep_dims=True)
print(np.allclose(res, tmp), res.shape)


Chunk up by 2000 shots
True (20, 100, 100)


Plus with `keep_dims=True`, `adaptive_contraction` can act so much like `np.tensordot(A, B, axes=(0,0))`. By default `keep_dims=False`, so the outcome is just the matrix.

In [8]:
tmp = np.tensordot(arrdum['A'][:,slca], arrdum['C'][:,:200,:200], axes=(0,0))
res = adaptive_contraction(lstdum['A'], lstdum['C'], a_slice=slca, b_slice=slcv)
print(np.allclose(res, tmp.reshape(2,-1)), res.shape)

Chunk up by 2000 shots
True (2, 10000)


### Unit Tests for `shots_asarray`
Although it's a critical subroutine for `adaptive_contraction`, it's not relevant to the usage of that function. Just ignore it if `adaptive_contraction` works fine.

In [92]:
arr = arrdum['A']
print(np.allclose(shots_asarray(arr, None, None), (arr)))
print(np.allclose(shots_asarray(arr, selecti, None), (arr[10:15])))
print(np.allclose(shots_asarray(arr, None, slca), (arr[:,10:14:2])))
print(np.allclose(shots_asarray(arr, selecti, slca), (arr[10:15,10:14:2])))

True
True
True
True


In [93]:
lst,arr = lstdum['B'], arrdum['B']
print(np.allclose(shots_asarray(lst, None, None), (arr)))
print(np.allclose(shots_asarray(lst, selecti, None), (arr[10:15])))
print(np.allclose(shots_asarray(lst, None, slcb), (arr[:,3:5,4:6])))
print(np.allclose(shots_asarray(lst, selecti, slcb), (arr[10:15,3:5,4:6])))
print(np.allclose(shots_asarray(lst, selecti, slcv), (arr[10:15,:,4:6])))

True
True
True
True
True


In [94]:
dct,arr = dctdum['C'], arrdum['C']
print(np.allclose(shots_asarray(dct, None, None), (arr)))
print(np.allclose(shots_asarray(dct, selectk, None), (arr[10:15])))
print(np.allclose(shots_asarray(dct, None, slcb), (arr[:,3:5,4:6])))
print(np.allclose(shots_asarray(dct, selectk, slcb), (arr[10:15,3:5,4:6])))
print(np.allclose(shots_asarray(dct, selectk, slcv), (arr[10:15,:,4:6])))

True
True
True
True
True
